In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

### This is the code used to filter features.
### To generate the "optiver_features.parquet" file simply run the submission notebook and output the initial "train" dataframe to a parquet file.

In [ ]:
!pip install BorutaShap

In [ ]:
df = pd.read_parquet('../input/optiver-features-2/optiver_features.parquet')

In [ ]:
# remove this line if no features are name trade_price_<lambda_0>, for some reason BarutaSHAP doesn't like that name
df['trade_lambda'] = df['trade_price_<lambda_0>']
df.drop(columns = ['trade_price_<lambda_0>'], inplace = True)

In [ ]:
X_train = df[[x for x in list(df) if x not in ['stock_id', 'time_id', 'target', 'row_id']]]
X_train.replace([np.inf, -np.inf], np.nan, inplace = True)
X_train = X_train.fillna(X_train.mean())
y_train = df['target']

#### Set n_trial to something in the range of 50-100 for optimal results, 20 was used here because of Kaggle's runtime limit.

In [ ]:
from BorutaShap import BorutaShap
from xgboost import XGBRegressor

# If no model is selected default is the Random Forest
# If classification is True it is a classification problem
Feature_Selector = BorutaShap(model = XGBRegressor(), importance_measure='shap', classification=False)

Feature_Selector.fit(X=X_train, y=y_train, n_trials=20, random_state=0)

In [ ]:
Feature_Selector.plot(which_features='all', figsize=(100,50))

In [ ]:
Feature_Selector.Subset()